# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Kafka Producer: Financial Transaction Generator** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team members**: 
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [1]:
import findspark
findspark.init()
#dbe4c53ffd27d3d2bdef46d2f12d349caadcfcb1438e112f

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://1c173c8c3406:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe0c0976-7fd5-4018-b9b9-90b41c139cfc;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

In [3]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "d4f3f3b43802:9093") \
                .option("subscribe", "kafka-spark-example-0") \
                .option("subscribe", "kafka-spark-example-1") \
                .option("subscribe", "kafka-spark-example-2") \
                .option("subscribe", "kafka-spark-example-3") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
from pyspark.sql.functions import split, col, expr

kafka_df = kafka_lines.select(split(col("value"), ",").alias("pairs_array"))

kafka_df = kafka_df.withColumn("user_id", split(col("pairs_array").getItem(0), ":").getItem(1))
kafka_df = kafka_df.withColumn("video_id", split(col("pairs_array").getItem(1), ":").getItem(1))
kafka_df = kafka_df.withColumn("watch_time_seconds", split(col("pairs_array").getItem(2), ":").getItem(1))
kafka_df = kafka_df.withColumn("resolution", split(col("pairs_array").getItem(3), ":").getItem(1))
kafka_df = kafka_df.withColumn("bitrate_kbps", split(col("pairs_array").getItem(4), ":").getItem(1))
kafka_df = kafka_df.withColumn("buffering_events", split(col("pairs_array").getItem(5), ":").getItem(1))
kafka_df = kafka_df.withColumn("paused", split(col("pairs_array").getItem(6), ":").getItem(1))
kafka_df = kafka_df.withColumn("skipped", split(col("pairs_array").getItem(7), ":").getItem(1))
kafka_df = kafka_df.withColumn("genre", split(col("pairs_array").getItem(8), ":").getItem(1))
kafka_df = kafka_df.withColumn("region", split(col("pairs_array").getItem(9), ":").getItem(1))
kafka_df = kafka_df.withColumn("recommended", split(col("pairs_array").getItem(10), ":").getItem(1))

# Usamos expr para hacer la resta de longitud directamente
kafka_df = kafka_df.withColumn(
    "timestamp",
    expr("substring(split(pairs_array[11], ':')[1], 1, length(split(pairs_array[11], ':')[1]) - 1)")
)

kafka_df.printSchema()


root
 |-- pairs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- user_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- watch_time_seconds: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- bitrate_kbps: string (nullable = true)
 |-- buffering_events: string (nullable = true)
 |-- paused: string (nullable = true)
 |-- skipped: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- region: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [5]:
query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)
query.stop()

25/05/10 21:57:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7de11b68-c138-4c18-ba7d-3020ee66b294. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/10 21:57:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/10 21:57:35 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+
|pairs_array|user_id|video_id|watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre|region|recommended|timestamp|
+-----------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+
+-----------+-------+--------+------------------+----------+------------+----------------+------+-------+-----+------+-----------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+--------------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                            |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre         |region|recommended|timestamp    |
+--------------------------------------------

-------------------------------------------
Batch: 4
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                      |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+------------------------------------------------------------------

-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                     |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+--------------------------------------------------------------------

...

-------------------------------------------
Batch: 75
-------------------------------------------
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                     |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+-------------------------------------------------------------------

-------------------------------------------
Batch: 76
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                      |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+-----------------------------------------------------------------

-------------------------------------------
Batch: 77
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                      |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+-----------------------------------------------------------------

-------------------------------------------
Batch: 79
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+--------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                  |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre   |region|recommended|timestamp    |
+---------------------------------------------------------------------------

-------------------------------------------
Batch: 81
-------------------------------------------
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+---------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                      |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre    |region|recommended|timestamp    |
+-----------------------------------------------------------------

In [ ]:
query_files = kafka_df \
                .writeStream \
                .format("parquet") \
                .mode("overwrite") \
                .option("path", "/home/jovyan/notebooks/data/project_parquet") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(300)
query_files.stop()

25/05/10 22:02:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/10 22:02:41 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [13]:
df = spark.read.parquet("/home/jovyan/notebooks/data/project_parquet")
df.show(5, False)


Py4JJavaError: An error occurred while calling o138.parquet.
: java.lang.IllegalStateException: /home/jovyan/notebooks/data/project_parquet/_spark_metadata/0 doesn't exist (latestId: 1, compactInterval: 10)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.$anonfun$allFiles$4(CompactibleFileStreamLog.scala:267)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.$anonfun$allFiles$2(CompactibleFileStreamLog.scala:265)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.$anonfun$allFiles$2$adapted(CompactibleFileStreamLog.scala:263)
	at scala.collection.StrictOptimizedIterableOps.flatMap(StrictOptimizedIterableOps.scala:118)
	at scala.collection.StrictOptimizedIterableOps.flatMap$(StrictOptimizedIterableOps.scala:105)
	at scala.collection.immutable.NumericRange.flatMap(NumericRange.scala:40)
	at org.apache.spark.sql.execution.streaming.CompactibleFileStreamLog.allFiles(CompactibleFileStre

In [7]:
sc.stop()